In [1]:
import data_preparation

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, regularizers, callbacks

df = data_preparation.CLEANED_TRAINING_DATA
test_df = data_preparation.CLEANED_TEST_DATA
original_test_df = pd.read_csv("data/test.csv")
test_ids = original_test_df.PassengerId

2024-01-19 22:38:47.740087: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-19 22:38:47.797037: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 22:38:47.797095: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 22:38:47.797118: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 22:38:47.809217: I tensorflow/core/platform/cpu_feature_g

In [2]:
df.head(1)

,Age,Embarked_C,Embarked_Q,Embarked_S,Fare,HasCabin,NameLength,Parch,Pclass,Sex,...,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir
0,0.271174,0,0,1,0.014151,0,0.15942,0,3,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
test_df.head(1)

,Age,Embarked_C,Embarked_Q,Embarked_S,Fare,HasCabin,NameLength,Parch,Pclass,Sex,...,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir
0,0.452723,0,1,0,0.015282,0,0.061224,0,3,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
df = data_preparation.dummify(df, "Survived")

In [5]:
tf.config.list_physical_devices('GPU')

2024-01-19 22:38:51.441888: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-19 22:38:51.450599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-19 22:38:51.450983: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
x = df.drop(columns=['Survived_0', 'Survived_1'])
y = df[['Survived_0', 'Survived_1']]

In [7]:
x.shape

(891, 28)

In [8]:
# Split made for validation purposes rather than testing, since the test set was given separately.

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [9]:
x_train.shape

(668, 28)

In [10]:
x_np = np.array([x], dtype=np.float32)
y_np = np.array([y], dtype=np.float32)

x_tf = tf.convert_to_tensor(x, dtype=tf.float32)
y_tf = tf.convert_to_tensor(y, dtype=tf.float32)

x_train_np = np.array([x_train], dtype=np.float32)
x_test_np = np.array([x_test], dtype=np.float32)
y_train_np = np.array([y_train], dtype=np.float32)
y_test_np = np.array([y_test], dtype=np.float32)

x_train_tf = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test_tf = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)

test_data_tf = tf.convert_to_tensor(test_df, dtype=tf.float32)



2024-01-19 22:38:51.528552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-19 22:38:51.529382: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-19 22:38:51.530002: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-19 22:38:51.941119: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-19 22:38:51.941444: I tensorflow/compile

In [11]:
print(x_train_tf.shape)
print(x_tf.shape)
print(test_data_tf.shape)

(668, 28)
(891, 28)
(418, 28)


In [12]:
model = models.Sequential()
model.add(layers.Input(shape=x_train_tf.shape[1:]))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2))

In [13]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics =['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1856      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1986 (7.76 KB)
Trainable params: 1986 (7.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
history = model.fit(x_train_tf,
                    y_train_tf,
                    epochs = 80)

Epoch 1/50


2024-01-19 22:38:53.256702: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-19 22:38:55.337351: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f3ee05220a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-19 22:38:55.337475: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2024-01-19 22:38:55.351189: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-19 22:38:55.391941: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-01-19 22:38:55.616654: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


21/21 [==============================] - 4s 18ms/step - loss: 3.8801 - accuracy: 0.6198
Epoch 2/50
21/21 [==============================] - 0s 15ms/step - loss: 3.3225 - accuracy: 0.6347
Epoch 3/50
21/21 [==============================] - 0s 13ms/step - loss: 2.5168 - accuracy: 0.6751
Epoch 4/50
21/21 [==============================] - 0s 11ms/step - loss: 2.1048 - accuracy: 0.6647
Epoch 5/50
21/21 [==============================] - 0s 11ms/step - loss: 1.8132 - accuracy: 0.6871
Epoch 6/50
21/21 [==============================] - 0s 10ms/step - loss: 1.8915 - accuracy: 0.7126
Epoch 7/50
21/21 [==============================] - 0s 10ms/step - loss: 2.1619 - accuracy: 0.6811
Epoch 8/50
21/21 [==============================] - 0s 9ms/step - loss: 1.8768 - accuracy: 0.7141
Epoch 9/50
21/21 [==============================] - 0s 7ms/step - loss: 1.7246 - accuracy: 0.7126
Epoch 10/50
21/21 [==============================] - 0s 7ms/step - loss: 1.7795 - accuracy: 0.7380
Epoch 11/50
21/21 [====

In [17]:
test_loss, test_acc = model.evaluate(x_test_tf, y_test_tf, verbose=2)

7/7 - 0s - loss: 0.6074 - accuracy: 0.7937 - 203ms/epoch - 29ms/step


In [18]:
probability_model = tf.keras.Sequential([model,
                                         tf.keras.layers.Softmax()])

In [19]:
predictions = probability_model.predict(x_test_tf)

7/7 [==============================] - 0s 7ms/step


In [20]:
predictions.shape

(223, 2)

In [21]:
predictions[0][0]

0.7185549

In [22]:
a = np.argmax(predictions[0][0])
print(a, type(a))

0 <class 'numpy.int64'>


In [23]:
y_test_tf[0]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 0.], dtype=float32)>

In [24]:
model2 = models.Sequential()
model2.add(layers.Input(shape=x_tf.shape[1:]))
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(2))

In [25]:
model2.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics =['accuracy'])

In [26]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                1856      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1986 (7.76 KB)
Trainable params: 1986 (7.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
history = model2.fit(x_tf,
                    y_tf,
                    epochs = 80)

Epoch 1/50


28/28 [==============================] - 1s 15ms/step - loss: 4.2895 - accuracy: 0.5253
Epoch 2/50
28/28 [==============================] - 0s 11ms/step - loss: 3.2559 - accuracy: 0.5645
Epoch 3/50
28/28 [==============================] - 0s 11ms/step - loss: 3.1757 - accuracy: 0.6083
Epoch 4/50
28/28 [==============================] - 0s 11ms/step - loss: 2.6449 - accuracy: 0.6633
Epoch 5/50
28/28 [==============================] - 0s 11ms/step - loss: 2.6628 - accuracy: 0.6554
Epoch 6/50
28/28 [==============================] - 0s 12ms/step - loss: 2.5791 - accuracy: 0.6824
Epoch 7/50
28/28 [==============================] - 0s 12ms/step - loss: 2.3330 - accuracy: 0.7037
Epoch 8/50
28/28 [==============================] - 0s 12ms/step - loss: 2.3494 - accuracy: 0.7116
Epoch 9/50
28/28 [==============================] - 0s 12ms/step - loss: 2.2665 - accuracy: 0.7351
Epoch 10/50
28/28 [==============================] - 0s 13ms/step - loss: 2.2315 - accuracy: 0.7082
Epoch 11/50
28/28 [=

In [28]:
probability_model2 = tf.keras.Sequential([model2,
                                         tf.keras.layers.Softmax()])

In [29]:
predictions = probability_model2.predict(test_data_tf)

 1/14 [=>............................] - ETA: 1s

14/14 [==============================] - 0s 4ms/step


In [40]:
preds = []
for i in predictions:
    if i[0] < i[1]:
        preds.append(1)
    else:
        preds.append(0)

In [41]:
ids = [i for i in test_ids]

In [43]:
result = pd.DataFrame({
    "PassengerId":ids,
    "Survived":preds
    })

In [44]:
result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [45]:
result.to_csv("result.csv", index=False)